In [1]:
from curvaturemodel import curvature as curv 
from curvaturemodel import edge as edg 
import pandas as pd
import os
import sys
sys.path.append('/Users/cmagri1/OneDrive - Johns Hopkins/git')
from EncodingModel_cm296 import utils as emutils


In [2]:
savepath = '../../../data-00/'

In [3]:
import numpy as np
from tqdm import tqdm
import torch
from PIL import Image
from torchvision.transforms import functional as transforms


def listdir(dir, path=True):
    files = os.listdir(dir)
    files = [f for f in files if f != '.DS_Store']
    files = sorted(files)
    if path:
        files = [os.path.join(dir, f) for f in files]
    return files

imagenet_mean = (0.485, 0.456, 0.406)
imagenet_std = (0.229, 0.224, 0.225)


def image_to_model(image,model):
    if isinstance(image, str):
        image = Image.open(image)
#         .convert('LAD')
        image = transforms.to_grayscale(image)
        image = transforms.resize(image, (96, 96))    # Or whatever resolution you want
        image = transforms.to_tensor(image)
        image = transforms.normalize(image, mean=(0.5,), std=(0.5,))
        with torch.no_grad():
            features = model(image.unsqueeze(dim=0)).squeeze(dim=0)
    return features



def compute_features_curvature(model, conditionsPath,WordInf =[]):
    #takes model and loads the features for that image, needs path to of files in directory

    conditions = listdir(conditionsPath)

    #If it's THINGS this ensures we only pick up images from List of 1470
    if len(WordInf) > 0:
        new_conditions = []
        for c in conditions:
            w = c.split('/')[-1]
            if w in WordInf:
                new_conditions.append(c)
        conditions = new_conditions

    condition_features = {}
    for c in tqdm(conditions):
        c_name = c.split('/')[-1]
        
        stimuli = listdir(c)
        
        #resize according to resolution and square the image
        features = [image_to_model(s,model) for s in stimuli]
        
        features = torch.stack(features)
        feats = features.mean(dim=0).cpu().numpy()
        condition_features[c_name] = feats
    return condition_features

In [10]:

#Compute DNN features
def object_by_feature(namemodel,keyword, savepath,WordList=[]):

    namefile = savepath+ keyword + '_' + namemodel 
        
    if os.path.isfile(namefile+".csv"): #if it's already saves
        
        print('loading file with '+ keyword+ ' stimuli features for ' + namemodel + ' model')
        features = pd.read_csv(namefile+".csv"  , sep=",", header=None, index_col=0)
        condition_features = {}
        for index, row in features.iterrows():
            condition_features[index] = row 
            
            
    else:
        
        if keyword is 'things':
            PathToImgs = '../../../../THINGSdataset/Main/images'
            WordList = pd.read_csv('../code-00-preprocessdataset/' + "KeptTHINGSInfo.txt", sep=",")['Word'].to_numpy()
        elif keyword is 'object2vec':
            PathToImgs = 'data-object2vec/stimuli'
        
    
        
        print('Computing '+ keyword+ ' stimuli features for ' + namemodel + ' model')
        #Specify the model
        if namemodel is 'Curvature':
            model = curv.CurvatureModel()
        elif namemodel is 'Edge':
            model = edg.EdgeModel()
            
        condition_features = compute_features_curvature(model, PathToImgs,WordInf = WordList)
#         print(condition_features)

        
#         pd.DataFrame(condition_features).transpose().to_csv(namefile+".csv", index = True, header=False)
        np.save(namefile,condition_features)
    return condition_features

In [11]:
#Compute features of CNN
#---------Object2Vect-----------------
model = 'Curvature'
keyword = 'things'
object2vec_features = object_by_feature(model,keyword,savepath)
    

  0%|          | 0/1470 [00:00<?, ?it/s]

Computing things stimuli features for Curvature model


  0%|          | 4/1470 [00:03<19:24,  1.26it/s]


KeyboardInterrupt: 

In [13]:
#Compute features of CNN
#---------Object2Vect-----------------
model = 'Edge'
keyword = 'things'
object2vec_features = object_by_feature(model,keyword,savepath)

  0%|          | 0/1470 [00:00<?, ?it/s]

Computing things stimuli features for Edge model


100%|██████████| 1470/1470 [15:05<00:00,  1.62it/s]


In [ ]:
ex